<a href="https://colab.research.google.com/github/Anot11/Python-package-for-dynamic-panel-model/blob/main/Dynamic_panel_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install linearmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.7 MB/s eta 0:00:00


In [2]:
pip install pydynpd

In [3]:
import pandas as pd
from  pydynpd import regression

In [4]:
import numpy.linalg      as la
import matplotlib.pyplot as plt
import numpy             as np
import pandas            as pd
import statsmodels.api   as sm

In [5]:
from linearmodels.panel           import PooledOLS, RandomEffects, compare
from linearmodels                 import PanelOLS
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
from statsmodels.stats.stattools  import durbin_watson
from scipy                        import stats





In [6]:
import seaborn as sns

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
df=pd.read_csv('/content/drive/MyDrive/group.csv')

In [9]:
df.head()

,ID,IQ,Race,Sex,Year,HghstGrdCompl,Inc,Age,Exp,Exp2,...,d06,d08,d10,d12,d14,d16,d18,dwhite,dblack,dfemale
0,15,109,3,1,1979,8,0.0,14,0,0,...,0,0,0,0,0,0,0,0,0,1
1,15,109,3,1,1980,9,0.0,15,0,0,...,0,0,0,0,0,0,0,0,0,1
2,15,109,3,1,1981,10,0.0,16,0,0,...,0,0,0,0,0,0,0,0,0,1
3,15,109,3,1,1982,11,0.0,17,0,0,...,0,0,0,0,0,0,0,0,0,1
4,15,109,3,1,1983,12,900.0,18,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
# Rename the columns in the DataFrame df
df.rename(columns={
  'HghstGrdCompl': 'S',
    'Inc': 'y',
    'Age': 'A',
    'Exp': 'X',
    'Exp2': 'X2',
    'Marital Sts': 'MS'
}, inplace=True)

# Check the new column names
print(df.columns)


Index(['ID', 'IQ', 'Race', 'Sex', 'Year', 'S', 'y', 'A', 'X', 'X2', 'MS',
       'd80', 'd81', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87', 'd88', 'd89',
       'd90', 'd91', 'd92', 'd93', 'd94', 'd96', 'd98', 'd00', 'd02', 'd04',
       'd06', 'd08', 'd10', 'd12', 'd14', 'd16', 'd18', 'dwhite', 'dblack',
       'dfemale'],
      dtype='object')


In [11]:
df['ln_y'] = df['y'].apply(lambda x: np.log(x))

df = np.clip(df, a_min=0, a_max=None)


In [12]:
df.columns

Index(['ID', 'IQ', 'Race', 'Sex', 'Year', 'S', 'y', 'A', 'X', 'X2', 'MS',
       'd80', 'd81', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87', 'd88', 'd89',
       'd90', 'd91', 'd92', 'd93', 'd94', 'd96', 'd98', 'd00', 'd02', 'd04',
       'd06', 'd08', 'd10', 'd12', 'd14', 'd16', 'd18', 'dwhite', 'dblack',
       'dfemale', 'ln_y'],
      dtype='object')

typical dynamic panel data model:
Arellano-Bond GMM Estimator GMM approach based on all available instruments.

\

$$\log(y_{it}) = \lambda_1 y_{i,t-1} + \lambda_2 y_{i,t-2} + \beta_1 S_{it} + \beta_2 X_{it} + \beta_3 X_{it}^2 + \beta_4 D + \epsilon$$



logy denotes income, S is years of schooling, and X is years of experience since finishing school. Therefore, the exponent on S can be considered the ‘return to education’ and the exponent on X the ‘ return to experience.’ D is a vector of control variables such as IQ, Sex, Race and Marital status

Part 1 is a list starting with the dependent variable, and followed by independent variables except time dummies. Given the model, part 1 is:

ln_y L1.ln_y L2.ln_y S X
In the command above, L is the lag operator as in L1.n (i.e.,) or L2.n for 2 lags of n (i.e.,).

In [13]:
command_str='ln_y L1.ln_y L2.ln_y S A X X2 IQ dwhite dblack dfemale  | gmm(ln_y, 2:3) gmm(S, 1:.) iv(X)'

# Example 1

In [14]:
#M1
mydpd  = regression.abond(command_str, df, ['ID', 'Year'])

 Dynamic panel-data estimation, two-step system GMM
 Group variable: ID                                Number of obs = 29700    
 Time variable: Year                               Min obs per group: 25    
 Number of instruments = 485                       Max obs per group: 25    
 Number of groups = 1188                           Avg obs per group: 25.00 
+---------+------------+---------------------+------------+-----------+-----+
|   ln_y  |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+---------+------------+---------------------+------------+-----------+-----+
| L1.ln_y | 0.6859741  |      0.0097242      | 70.5428518 | 0.0000000 | *** |
| L2.ln_y | 0.1483826  |      0.0091609      | 16.1973964 | 0.0000000 | *** |
|    S    | 5.7672645  |      2.9755864      | 1.9381943  | 0.0525995 |     |
|    A    | -5.7284964 |      2.9761631      | -1.9247925 | 0.0542553 |     |
|    X    | 5.6637338  |      2.9785843      | 1.9014851  | 0.0572385 |     |
|    X2   | 0.00

The result shows that the regression is a two-step system GMM, which is the default setting because we didn't include part 3 in our command string. There are 1188 individuals in the imbalanced sample over 25 (=27-2) years as we include the second lag of the dependent variable (i.e., L2.n).

Hansen over-identification test is significant, which means that our choices of instruments are not exogenous.

Because the regression doesn't pass Hansen over-identification test, we change our assumptions to: Both $S$ and $X$ are predetermined variables

#Example 2

**Example 2**
Then we modify our code as follows. Note that "gmm (S X, 1:.) iv(X)" is changed to "gmm(S X, 1:.)" based on the new assumption.

We also remove variables that were not significant in the previous regression

In [15]:
#M2
command_str='ln_y L1.ln_y L2.ln_y S X X2 dwhite dblack | gmm(ln_y, 2:3) gmm(S X2, 1:.)'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])

 Dynamic panel-data estimation, two-step system GMM
 Group variable: ID                                Number of obs = 29700    
 Time variable: Year                               Min obs per group: 25    
 Number of instruments = 882                       Max obs per group: 25    
 Number of groups = 1188                           Avg obs per group: 25.00 
+---------+------------+---------------------+-------------+-----------+-----+
|   ln_y  |   coef.    | Corrected Std. Err. |      z      |   P>|z|   |     |
+---------+------------+---------------------+-------------+-----------+-----+
| L1.ln_y | 0.6858527  |      0.0086484      |  79.3037279 | 0.0000000 | *** |
| L2.ln_y | 0.1485237  |      0.0078121      |  19.0120568 | 0.0000000 | *** |
|    S    | 0.0464723  |      0.0099476      |  4.6717200  | 0.0000030 | *** |
|    X    | -0.0744152 |      0.0048427      | -15.3665035 | 0.0000000 | *** |
|    X2   | 0.0011897  |      0.0001245      |  9.5525024  | 0.0000000 | *** |
|  dwhit

The model passes Arellano-Bond tests, but not the Hansen 😞

#Example 3

Apart from gmm() and iv(), we can also use endo() and pred() for convinience in part 2. endo(list of variables) is equivelent to gmm(list of variables, 2:.) while pred(list of variables) is the same as gmm(list of variables, 1:.).

For example, the code for the second model can be changed as follows and we get the same result.

In [16]:
#M3
command_str='ln_y L1.ln_y L2.ln_y S X X2 dwhite dblack | gmm(ln_y, 2:3) pred(S X2)'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])


 Dynamic panel-data estimation, two-step system GMM
 Group variable: ID                                Number of obs = 29700    
 Time variable: Year                               Min obs per group: 25    
 Number of instruments = 882                       Max obs per group: 25    
 Number of groups = 1188                           Avg obs per group: 25.00 
+---------+------------+---------------------+-------------+-----------+-----+
|   ln_y  |   coef.    | Corrected Std. Err. |      z      |   P>|z|   |     |
+---------+------------+---------------------+-------------+-----------+-----+
| L1.ln_y | 0.6858527  |      0.0086484      |  79.3037279 | 0.0000000 | *** |
| L2.ln_y | 0.1485237  |      0.0078121      |  19.0120568 | 0.0000000 | *** |
|    S    | 0.0464723  |      0.0099476      |  4.6717200  | 0.0000030 | *** |
|    X    | -0.0744152 |      0.0048427      | -15.3665035 | 0.0000000 | *** |
|    X2   | 0.0011897  |      0.0001245      |  9.5525024  | 0.0000000 | *** |
|  dwhit

# Part 2

We can change the default settings in part 3 of our command string. Part 2 includes the following possible options:

onestep: perform one-step GMM estimation rather than the default two-step GMM estimation. nolevel: only perform difference GMM iterated: perform iterative GMM estimation. timedumm: automatically include time dummies in part 1, and IV statement in part 2. hqic: if there are multiple competing models, let them be sorted by hqic rather than by the default bic. collapse: collapse instruments to reduce the proeblem of too many instruments

**Example 4** For example, we can change the regression to a one-step difference GMM by adding part 3 in our command string. Also, suppose we want to include more lagged dependent variables as instruments. That is, we want to use all available lagged rather than just L2.n and L3.ln_y in the previous models. So, we change gmm(Ln_y, 2:3) to gmm(Ln_y, 2:.) which is then simplified to endo(ln_y).

In [17]:
#M4
command_str='ln_y L1.ln_y L2.ln_y S X X2 dwhite dblack | endo(ln_y) pred(S X2) | onestep nolevel'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])


 Dynamic panel-data estimation, one-step difference GMM
 Group variable: ID                                 Number of obs = 29700    
 Time variable: Year                                Min obs per group: 25    
 Number of instruments = 1103                       Max obs per group: 25    
 Number of groups = 1188                            Avg obs per group: 25.00 
+---------+------------+---------------------+------------+-----------+-----+
|   ln_y  |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+---------+------------+---------------------+------------+-----------+-----+
| L1.ln_y | 0.5501162  |      0.0100973      | 54.4816548 | 0.0000000 | *** |
| L2.ln_y | 0.0688908  |      0.0077131      | 8.9317132  | 0.0000000 | *** |
|    S    | 0.0588886  |      0.0440400      | 1.3371593  | 0.1811706 |     |
|    X    | -0.0496375 |      0.0075165      | -6.6038418 | 0.0000000 | *** |
|    X2   | 0.0002814  |      0.0001707      | 1.6483389  | 0.0992832 |     |
|  dwhit

Example 5: Time Dummy Variables We can also add time dummy variables to the previous model:

In [18]:
#M5
command_str='ln_y L1.ln_y L2.ln_y S X X2 dwhite dblack | endo(ln_y) pred(S X2) | onestep nolevel timedumm'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])

 Dynamic panel-data estimation, one-step difference GMM
 Group variable: ID                                 Number of obs = 29700    
 Time variable: Year                                Min obs per group: 25    
 Number of instruments = 1128                       Max obs per group: 25    
 Number of groups = 1188                            Avg obs per group: 25.00 
+-----------+------------+---------------------+------------+-----------+-----+
|    ln_y   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+-----------+------------+---------------------+------------+-----------+-----+
|  L1.ln_y  | 0.5481622  |      0.0103811      | 52.8037582 | 0.0000000 | *** |
|  L2.ln_y  | 0.0679396  |      0.0079779      | 8.5159935  | 0.0000000 | *** |
|     S     | 0.0659673  |      0.1110018      | 0.5942901  | 0.5523181 |     |
|     X     | -0.0283532 |      0.0889932      | -0.3185992 | 0.7500304 |     |
|     X2    | -0.0005809 |      0.0003456      | -1.6810684 | 0.0927496 

As shown above, 28 dummy variables (year_1982 ~ year_2018) are added to the model and each of them is automatically treated as a instrument. That is why the number of instruments is increased from 11031128 in the previous model to 1128.

Reducing the number of instruments As indicated in the technique guide, too many instruments causes problems. There are two ways to reduce the number of instruments. The first is to curtail the number of instruments in gmm list. For example, in example 5, we used the following command string:

command_str='ln_y L1.ln_y S X | endo(ln_y) pred(S X) | onestep nolevel timedumm'

In its second part, endo(ln_y) is the same as gmm(ln_y, 2:.). In other words, we use all available lagged variable as long as it is deeper than the second lag. We can place a restriction by changing endo(ln_y) to gmm(ln_y, 2:4), which says only L2.ln_y, L3.ln_y, and L4.ln_y are used as instruments.

Example 6

In [19]:
#M6
command_str='ln_y L1.ln_y L2.ln_y S X X2 dwhite dblack | gmm(ln_y, 2:4) pred(S X2) | onestep nolevel timedumm'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])

 Dynamic panel-data estimation, one-step difference GMM
 Group variable: ID                                Number of obs = 29700    
 Time variable: Year                               Min obs per group: 25    
 Number of instruments = 852                       Max obs per group: 25    
 Number of groups = 1188                           Avg obs per group: 25.00 
+-----------+------------+---------------------+------------+-----------+-----+
|    ln_y   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+-----------+------------+---------------------+------------+-----------+-----+
|  L1.ln_y  | 0.5273181  |      0.0112532      | 46.8592200 | 0.0000000 | *** |
|  L2.ln_y  | 0.0495567  |      0.0084510      | 5.8639791  | 0.0000000 | *** |
|     S     | 0.2186730  |      0.1188114      | 1.8405051  | 0.0656941 |     |
|     X     | 0.0636638  |      0.0927850      | 0.6861429  | 0.4926230 |     |
|     X2    | -0.0007282 |      0.0003759      | -1.9374305 | 0.0526927 |   

As you can see, the number of instruments is decreased from 1128  in the previous example to 852.

The second way to reduce the number of instruments is to add "collapse" to part 3. In the next example, we just add "collapse" to the model in example 5.

In [20]:
#M7
command_str='ln_y L1.ln_y L2.ln_y S X X2 dwhite dblack | endo(ln_y) pred(S X2) | onestep nolevel timedumm collapse'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])

 Dynamic panel-data estimation, one-step difference GMM
 Group variable: ID                                Number of obs = 29700    
 Time variable: Year                               Min obs per group: 25    
 Number of instruments = 108                       Max obs per group: 25    
 Number of groups = 1188                           Avg obs per group: 25.00 
+-----------+-------------+---------------------+------------+-----------+-----+
|    ln_y   |    coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+-----------+-------------+---------------------+------------+-----------+-----+
|  L1.ln_y  |  0.5412756  |      0.0122866      | 44.0542874 | 0.0000000 | *** |
|  L2.ln_y  |  0.0672045  |      0.0087703      | 7.6627762  | 0.0000000 | *** |
|     S     |  0.5642435  |      0.3090660      | 1.8256411  | 0.0679043 |     |
|     X     |  0.3281434  |      0.2115255      | 1.5513182  | 0.1208254 |     |
|     X2    |  -0.0001792 |      0.0005691      | -0.3149747 | 0.7527

An innovative feature of pydynpd is that it can try all possible lags and return models that pass Hansen and AR(2) tests.

For example, suppose we want the system to try different models with different numbers of lagged dependent variable . We can indicate this request by using "L(1:?).ln_y" in part 2. The question mark means the system is requested to try L(1:1).ln_y, then L(1:2).ln_y, then L(1:3).ln_y, so on and so forth. Each attempt is a candidate model. The system will automatically filter out models and only show "good" models that satisfy all of the conditions below:

pass Hansen over-identification test (i.e., its P value should be greater than 5%) pass AR(2) test (i.e., its P value is greater than 5%) The P-value of Hansen over-identification test shouldn't be too high. In other words, it should be less than 99.99%. This is because a P value too close to 1 indicates a potential too-many-instrument issue.

Example 8: automatica mode (1) In this example, we let the system try different models by increasing the number of lagged dependent variable one by one

# Part 3 Automatic mode

In [21]:
#M8
command_str='ln_y L(1:?).ln_y S X X2 dwhite dblack | gmm(ln_y, 2:3) pred(S X2)'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])

 m16
 Dynamic panel-data estimation, two-step system GMM
 Group variable: ID                                Number of obs = 13068    
 Time variable: Year                               Min obs per group: 11    
 Number of instruments = 546                       Max obs per group: 11    
 Number of groups = 1188                           Avg obs per group: 11.00 
+----------+------------+---------------------+------------+-----------+-----+
|   ln_y   |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+----------+------------+---------------------+------------+-----------+-----+
| L1.ln_y  | 0.6877691  |      0.0163243      | 42.1315004 | 0.0000000 | *** |
| L2.ln_y  | 0.1022449  |      0.0162492      | 6.2923191  | 0.0000000 | *** |
| L3.ln_y  | 0.0813378  |      0.0348915      | 2.3311657  | 0.0197446 |  *  |
| L4.ln_y  | 0.0532899  |      0.0483810      | 1.1014634  | 0.2706950 |     |
| L5.ln_y  | -0.0062452 |      0.0328391      | -0.1901770 | 0.8491704 |     |
| L

Output above consists of four parts:

the regression table of each individual "good" model side by side comparision of these good models Andrews-Lu MMSC Criteria (note) of these good models. They are sorted by BIC list of models that didn't pass specification tests (note) Andrews, Donald WK, and Biao Lu. "Consistent model and moment selection procedures for GMM estimation with application to dynamic panel data models." Journal of Econometrics 101.1 (2001): 123-164. doi: 10.1016/S0304-4076(00)00077-4

As shown above, the system reports 9 models (models m17 ~ m24). Under the name of each model, its command string is displayed which allows users to replicate the output of the model.

The results of 9 of them are displayed because they all satisfy the conditions indicated above. On the other hand, models m1-m15 have not output because each of them doesn't satisfy at least one of the conditions.



Example 9: automatic mode (2) Question mark is not restricted to lagged dependent variable n . For example, we can try the following model with two question marks, one for variable Ln_y and the other for variable S.Then the system will try all combinations of lagged Ln_y and Ln_y lagged ,resulting in much more candidate models. Note that this time, we added option "hqic" to our model, which indicates that competing models would be sorted by HQIC, rather than BIC.

In [ ]:
#M9
command_str='ln_y L(1:?).ln_y dwhite dblack L1.S L2.S L(3:?).S X X2| gmm(ln_y, 2:3) pred(X X2)|hqic'

mydpd  = regression.abond(command_str, df, ['ID', 'Year'])